# 1. Definition of ```biobeam```-simulation

Deterministic parameters for the simulation

In [ ]:
import os
import json
import numpy as np

param_save_path = '/Volumes/davidh-ssd/mv-sim/sim_tissue/bbeam_params.json'
params = {}

### save path formatstring for simulated images 
# has to contain the following placeholders:
# {x} := index of x tile
# {y} := index of y tile
# {z} := index of z tile
# {illum} := 1=left/2=right
# {lam} := wavelength
save_fstring = params['save_fstring'] = '/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-bbeam-x{x}-y{y}-z{z}-i{illum}-l{lam}/'

seed = params['seed'] = 1222
raw_data_path = params['raw_data_path'] = '/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-phantom/'
raw_data_dims = params['raw_data_dims'] = (255, 2048, 2048)

# NB: only use downsampling for quick, local testing, do final sims on raw resolution!
downsampling = params['downsampling'] = 4

# NB: z should not be subsampled so much (psz should be ~lambda/2)
# TODO: re-do with larger volume
#phys_dims = params['phys_dims'] = (400, 400, 50)
phys_dims = params['phys_dims'] = (400, 400, 200)

na_illum = params['na_illum'] = 0.2
na_detect = params['na_detect'] = 0.5
ri_medium =  params['ri_medium'] = 1.33
ri_delta_range = params['ri_delta_range'] = (.05, .1)
two_sided_illum = params['two_sided_illum'] = True
lambdas = params['lambdas'] = (400, )

fov_size = params['fov_size'] = (200, 450, 450)
point_fov_size = params['point_fov_size'] = (150, 400, 400)
n_points_per_fov = params['n_points_per_fov'] = 10
points_min_distance = params['points_min_distance'] = 20

min_off = params['min_off'] = (-10, -10, -10)
max_off = params['max_off'] = (10, 10, 10)

#x_locs = params['x_locs'] = (1024 - 600, 1024 - 200, 1024 + 200, 1024 + 600)
x_locs = params['x_locs'] = (1024 - 600, 1024 - 200,)
#y_locs = params['y_locs'] = (1024 - 600, 1024 - 200, 1024 + 200, 1024 + 600)
y_locs = params['y_locs'] = (1024 - 600, )

z_locs = params['z_locs'] = (128, )

padding = params['padding'] = 32


#os.path.relpath(raw_data_path, os.path.split(param_save_path)[0])
#params
#raw_data_path

# 2. Random offsets and descriptors

Generating random offsets of tiles and descriptors in each tile

In [ ]:
from sim_util import random_points_in_volume_min_distance

np.random.seed(seed)
fields = params['fields'] = {}

xidx, yidx, zidx = np.meshgrid(range(len(x_locs)), range(len(y_locs)), range(len(z_locs)))
for xi, yi, zi in zip(xidx.flat, yidx.flat, zidx.flat):
    #print(xi, yi, zi)
    
    off = [np.random.randint(min_off[i], max_off[i] + 1) for i in range(len(min_off))]
    loc = [z_locs[zi], y_locs[yi], x_locs[xi]]
    min_ = np.array(loc) - np.ceil(np.array(fov_size) / 2)
    max_ = np.array(loc) + np.floor(np.array(fov_size) / 2)
    
    min_points = min_ + (np.array(fov_size) - np.array(point_fov_size)) / 2
    max_points = min_ + (np.array(fov_size) + np.array(point_fov_size)) / 2
    
    points = random_points_in_volume_min_distance(min_points, max_, points_min_distance, n_points_per_fov)
    
    fields[','.join(map(str, (xi, yi, zi)))] = {}
    fields[','.join(map(str, (xi, yi, zi)))]['off'] = off
    fields[','.join(map(str, (xi, yi, zi)))]['points'] = [list(map(int, p)) for p in list(points)]
    

# Save simulation params

In [ ]:
with open(param_save_path, 'w') as fd:
    json.dump(params, fd, indent=1)

# Run simulation, better to do this via run_sim2.py

In [ ]:
import gputools
import pyopencl

# set and check which gpu is used
gputools.init_device(id_platform = 0, id_device = 1)
gputools.get_device()

In [ ]:
from sim_util import sim_from_definition

param_save_path = '/Volumes/davidh-ssd/mv-sim/sim_tissue/bbeam_params.json'
sim_from_definition(param_save_path)

# Misc. stuff

In [ ]:
# Approximately Normalize and scale to desired SNR
# TODO: extract to function

img = load_tiff_sequence('/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-bbeam-x0-y0-z0-iright-l400/', 'bbeam_sim_c1.*')

snr = 7
q = 0.98
img = img / np.quantile(img, q) * snr**2

save_as_sequence(img, '/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-bbeam-x0-y0-z0-iright-l400/', file_pattern='bbeam_sim_c1_z{plane}.tif')
